In [1]:
!python -V

Python 3.11.5


In [2]:
import pandas as pd

In [3]:
import os
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
#from sklearn.metrics import root_mean_squared_error

In [7]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('file:///C:/Users/Cokroaminoto/Documents/Analisis Data/Data '
 'Scientist/DataTalksClub/MLOps - zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1727228916584, experiment_id='1', last_update_time=1727228916584, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
#df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
#df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

df_train = read_dataframe('/datasets/dtc/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/datasets/dtc/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715206838056

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
# Create directories if they don't exist
#os.makedirs('/models/', exist_ok=True)  # Create all subdirectories

try:
    os.makedirs('models/', exist_ok=True)
except Exception as e:
    print(f"Error creating models folder: {e}")

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "adi")

    mlflow.log_param("train-data-path", "/datasets/dtc/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "/datasets/dtc/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            #num_boost_round=1000,  # for processor multi-core Intel i5 with 8GB RAM or higher
            num_boost_round=200,  # Reduced from 1000 for processor Intel Celeron N3350 with 4GB RAM 
            evals=[(valid, 'validation')],
            early_stopping_rounds=50  # for processor multi-core Intel i5 with 8GB RAM or higher 
            early_stopping_rounds=20  # Reduced from 50 for processor Intel Celeron N3350 with 4GB RAM 
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}


In [20]:
search_space = {
    #'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # for processor multi-core Intel i5 with 8GB RAM or higher
    'max_depth': scope.int(hp.quniform('max_depth', 4, 20, 1)),  # Reduced range for processor Intel Celeron N3350 with 4GB RAM 
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    #'objective': 'reg:linear', # for processor multi-core Intel i5 with 8GB RAM or higher
    'objective': 'reg:squarederror',  # Updated objective for processor Intel Celeron N3350 with 4GB RAM 
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:11.57613                                                                                           
[1]	validation-rmse:11.00614                                                                                           
[2]	validation-rmse:10.49722                                                                                           
[3]	validation-rmse:10.04517                                                                                           
[4]	validation-rmse:9.64335                                                                                            
[5]	validation-rmse:9.28717                                                                                            
[6]	validation-rmse:8.97251                                                                                            
[7]	validation-rmse:8.69515                                                                                            
[8]	validation-rmse:8.45219             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:6.79940                                                                                            
[1]	validation-rmse:6.73880                                                                                            
[2]	validation-rmse:6.72772                                                                                            
[3]	validation-rmse:6.71758                                                                                            
[4]	validation-rmse:6.70505                                                                                            
[5]	validation-rmse:6.69519                                                                                            
[6]	validation-rmse:6.69018                                                                                            
[7]	validation-rmse:6.68682                                                                                            
[8]	validation-rmse:6.68422             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:10.60909                                                                                           
[1]	validation-rmse:9.44586                                                                                            
[2]	validation-rmse:8.62179                                                                                            
[3]	validation-rmse:8.04536                                                                                            
[4]	validation-rmse:7.64744                                                                                            
[5]	validation-rmse:7.37443                                                                                            
[6]	validation-rmse:7.18688                                                                                            
[7]	validation-rmse:7.05921                                                                                            
[8]	validation-rmse:6.96942             

...

[0]	validation-rmse:9.28873                                                                                            
[1]	validation-rmse:7.88242                                                                                            
[2]	validation-rmse:7.24405                                                                                            
[3]	validation-rmse:6.95322                                                                                            
[4]	validation-rmse:6.82004                                                                                            
[5]	validation-rmse:6.75493                                                                                            
[6]	validation-rmse:6.71686                                                                                            
[7]	validation-rmse:6.69885                                                                                            
[8]	validation-rmse:6.68728             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:9.94657                                                                                            
[1]	validation-rmse:8.56184                                                                                            
[2]	validation-rmse:7.74689                                                                                            
[3]	validation-rmse:7.28168                                                                                            
[4]	validation-rmse:7.01675                                                                                            
[5]	validation-rmse:6.86417                                                                                            
[6]	validation-rmse:6.77140                                                                                            
[7]	validation-rmse:6.71506                                                                                            
[8]	validation-rmse:6.68189             

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|█████████████████████████████████████████████████| 50/50 [58:37<00:00, 70.36s/trial, best loss: 6.349295728606375]


In [21]:
mlflow.xgboost.autolog(disable=True)

In [25]:
'''
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
'''

with mlflow.start_run():
    best_result['max_depth'] = int(best_result['max_depth'])  # Ensure max_depth is an integer
    mlflow.log_params(best_result)
    booster = xgb.train(
        params=best_result,
        dtrain=train,
        #num_boost_round=1000, # for processor multi-core Intel i5 with 8GB RAM or higher
        num_boost_round=200,  # Reduced from 1000 for processor Intel Celeron N3350 with 4GB RAM 
        evals=[(valid, 'validation')],
        #early_stopping_rounds=50 # for processor multi-core Intel i5 with 8GB RAM or higher
        early_stopping_rounds=20  # Reduced from 50 for processor Intel Celeron N3350 with 4GB RAM 
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:8.11863
[1]	validation-rmse:7.03517
[2]	validation-rmse:6.75099
[3]	validation-rmse:6.65674
[4]	validation-rmse:6.62207
[5]	validation-rmse:6.60615
[6]	validation-rmse:6.60176
[7]	validation-rmse:6.59413
[8]	validation-rmse:6.59027
[9]	validation-rmse:6.58789
[10]	validation-rmse:6.57710
[11]	validation-rmse:6.57294
[12]	validation-rmse:6.56953
[13]	validation-rmse:6.56571
[14]	validation-rmse:6.56188
[15]	validation-rmse:6.55840
[16]	validation-rmse:6.55278
[17]	validation-rmse:6.54859
[18]	validation-rmse:6.54424
[19]	validation-rmse:6.53830
[20]	validation-rmse:6.53473
[21]	validation-rmse:6.53268
[22]	validation-rmse:6.53031
[23]	validation-rmse:6.52552
[24]	validation-rmse:6.52114
[25]	validation-rmse:6.51677
[26]	validation-rmse:6.51223
[27]	validation-rmse:6.50935
[28]	validation-rmse:6.50744
[29]	validation-rmse:6.50425
[30]	validation-rmse:6.50151
[31]	validation-rmse:6.50006
[32]	validation-rmse:6.49816
[33]	validation-rmse:6.49490
[34]	validation-rmse:6.4

C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:35:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/09/25 19:38:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [26]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/datasets/dtc/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "/datasets/dtc/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/09/25 19:48:55 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
C:\Users\Cokroaminoto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/25 20:58:27 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/09/25 21:01:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\COKROA~1\AppData\Local\Temp\tmp3s7adk41\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.4.1.post1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
C:\Users\Cokroaminoto\AppDa

KeyboardInterrupt: 